In [ ]:
from utils import input_pipeline
from utils import MMCombiner
from utils import PCTCombiner
from utils import PSCombiner

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import dill as pkl
import os

import torch
import torch.nn as nn

gem_classifiers = pkl.load(open(os.path.join('models', 'gem_classifiers.pkl'), 'rb'))

/shared-libs/python3.9/py/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
data_np = np.load(os.path.join('data','task_4_test_dd4bd32b08b776e6_daf99ad_numpy.pkl'), allow_pickle=True)
data_pd = pd.read_pickle(os.path.join('data','task_4_test_dd4bd32b08b776e6_daf99ad_pandas.pkl'))
data_pd = data_pd.reset_index()

ids = data_pd[['pianist_id', 'segment_id', 'snippet_id']]

features = data_pd.iloc[:,4:]
features = features.reset_index()
features = features.drop(columns='index')

X = input_pipeline.fit_transform(features)

scaler = StandardScaler()
X_for_gem = scaler.fit_transform(X)

In [ ]:
# Predict GEM-features

for clf in gem_classifiers.values():
    X = np.c_[X, clf.predict(X_for_gem)]

scaler_total = StandardScaler()
X = scaler_total.fit_transform(X)

In [ ]:
"""
class nn_classifier(nn.Module):
    def __init__(self, n_features=99):
        super(nn_classifier, self).__init__()

        self.input_layer = nn.Linear(n_features, 128)
        self.input_activation = nn.SELU()

        self.hidden_layers = nn.Sequential(
            nn.Linear(128, 64),
            nn.AlphaDropout(p=0.1),
            nn.SELU(),
            nn.Linear(64, 32),
            nn.AlphaDropout(p=0.1),
            nn.Linear(32, 4)
        )

        self.output_layer = nn.Softmax(1)

    def forward(self, x):
        x = self.input_activation(self.input_layer(x))
        x = self.hidden_layers(x)
        output = self.output_layer(x)
        return output

model = torch.load(os.path.join('models', 'model1.pt'))
"""

"\nclass nn_classifier(nn.Module):\n    def __init__(self, n_features=99):\n        super(nn_classifier, self).__init__()\n\n        self.input_layer = nn.Linear(n_features, 128)\n        self.input_activation = nn.SELU()\n\n        self.hidden_layers = nn.Sequential(\n            nn.Linear(128, 64),\n            nn.AlphaDropout(p=0.1),\n            nn.SELU(),\n            nn.Linear(64, 32),\n            nn.AlphaDropout(p=0.1),\n            nn.Linear(32, 4)\n        )\n\n        self.output_layer = nn.Softmax(1)\n\n    def forward(self, x):\n        x = self.input_activation(self.input_layer(x))\n        x = self.hidden_layers(x)\n        output = self.output_layer(x)\n        return output\n\nmodel = torch.load(os.path.join('models', 'model1.pt'))\n"

In [ ]:
class nn_classifier2(nn.Module):
    def __init__(self, n_features=99):
        super(nn_classifier2, self).__init__()

        self.input_layer = nn.Linear(n_features, 256)
        self.input_activation = nn.SELU()

        self.hidden_layers = nn.Sequential(
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.AlphaDropout(p=0.3),
            nn.SELU(),

            nn.Linear(128, 64),
            nn.BatchNorm1d(64),
            nn.AlphaDropout(p=0.3),
            nn.SELU(),

            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.AlphaDropout(p=0.3),
            nn.SELU(),

            nn.Linear(32, 4)
        )

        self.output_layer = nn.Softmax(1)

    def forward(self, x):
        x = self.input_activation(self.input_layer(x))
        x = self.hidden_layers(x)
        output = self.output_layer(x)
        return output

model = torch.load(os.path.join('models', 'best_model_loss_2.pt'))

In [ ]:
class nn_classifier3(nn.Module):
    def __init__(self, n_features, n_layers,hidden_size,dropout_rate):
        super(nn_classifier3, self).__init__()

        self.net = nn.Sequential()
        self.net.add_module('in_lin', nn.Linear(n_features, hidden_size))
        self.net.add_module('in_norm',  nn.BatchNorm1d(hidden_size))
        self.net.add_module('in_act', nn.ReLU())
        self.net.add_module('in_drop',nn.Dropout(dropout_rate))

        for i in range(n_layers):
            self.net.add_module(f'hidden_lin_{i}', nn.Linear(hidden_size, hidden_size))
            self.net.add_module(f'hidden_norm_{i}',  nn.BatchNorm1d(hidden_size))
            self.net.add_module(f'hidden_act_{i}', nn.ReLU())
            self.net.add_module(f'hidden_drop_{i}',nn.Dropout(dropout_rate))

        self.net.add_module('out_lin', nn.Linear(hidden_size, 4))
        self.net.add_module('out_norm',  nn.Softmax(1))

    def forward(self, x):
        return self.net(x)

In [ ]:
model = torch.load('models/final_model.pt')

In [ ]:
output = model(torch.Tensor(X))
winners = output.argmax(dim=1) + 1

In [ ]:
ids['quadrant'] = winners

<ipython-input-7-bdb1de3a774b>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ids['quadrant'] = winners


In [ ]:
ids.to_csv(os.path.join('predictions', 'Finale_Abgabe.csv'), index=False)

In [ ]:
len(ids)

2085

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=e2905743-bdaf-45dd-a896-9824e6125426' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>